In [60]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

s = pd.read_csv('../data/taxi-distance.csv', header=None).squeeze()

# Beyond 1

Compare the mean and median trip distances. What does that tell you about the distribution of our data?

여행 거리(trip distance)의 평균(mean)과 중앙값(median)을 비교하시오.
이를 통해 데이터 분포에 대해 무엇을 알 수 있는가?

In [61]:
s.describe()
#describe()로 평균과 중앙값을 확인 가능
#평균은 3.158511, 
#중앙값은 1.7

count    9999.000000
mean        3.158511
std         4.037516
min         0.000000
25%         1.000000
50%         1.700000
75%         3.300000
max        64.600000
Name: 0, dtype: float64

Because the mean is significantly higher than the median, it would seem that there are some *very* long trips in our data set that are pulling the mean up. And sure enough, we see that the standard deviation is 4, but that we have at least one trip > 64 miles in length.

평균이 중앙값보다 훨씬 크다는 것은, 데이터셋에 매우 긴 여행 거리(극단치)가 존재하여 평균을 끌어올리고 있음을 의미한다. 실제로 표준편차가 4인데도, 최소 한 번의 여행 거리가 64마일 이상임을 확인할 수 있다.
최소 한 번의 여행 거리가 64마일 이상임을 확인할 수 있다는 것은 최대값으로 64가 존재한다는 것을 의미.

표준편차가 4라는 것은 평균에서 데이터가 보통 +- 4 의 범위에 존재한다는 것임. 좀 더 차이가 날 수 있지만, 평균과의 차이가 61정도 나는 것은 이상하다고 볼 수 있음.


# Beyond 2

How many short, medium, and long trips were there for trips that had only one passenger? Note that data for passenger count and trip length are from the same data set, meaning that the indexes are the same.

승객이 1명만 탑승한 여행 가운데, 짧은 여행·중간 여행·긴 여행은 각각 몇 건이 있었는가?
(승객 수와 여행 거리 데이터는 동일한 데이터셋에서 가져온 것이므로, 인덱스가 동일함을 유의하라.)

In [62]:
passenger_count = pd.read_csv('../data/taxi-passenger-count.csv', header=None).squeeze()

pd.cut(s[passenger_count == 1], 
       bins=[s.min(), 2, 10, s.max()], 
       include_lowest=True,
       labels=['short', 'medium', 'long']).value_counts()


# s시리즈에서 passenger_count가 1인 데이터만 추출하여, s시리즈 값 중 최소값부터 2까지, 2부터 10까지, 10부터 최대값까지 구간을 나누어 'short', 'medium', 'long'으로 라벨링
#include_lowest 옵션을 True로 설정하여 첫 구간에 최소값 포함
#각 구간에 속하는 데이터 개수를 value_counts()로 계산

0
short     4333
medium    2387
long       487
Name: count, dtype: int64

# Beyond 3

What happens if we don't pass explicit intervals, and instead ask `pd.cut` to just create 3 bins, with `bins=3`?

명시적인 구간(intervals)을 지정하지 않고, 단순히 bins=3을 사용하여 pd.cut으로 구간을 3개로 나누면 어떤 일이 발생하는가?

In [63]:
passenger_count = pd.read_csv('../data/taxi-passenger-count.csv', header=None).squeeze()

pd.cut(s[passenger_count == 1], 
       bins=3,
       labels=['short', 'medium', 'long']).value_counts()
#retbins를 True로 사용하면 튜플로 반환되기에 value_counts()를 사용할 수 없음

#s시리즈에서 passenger_count가 1인 데이터만 추출하여, s시리즈 값을 3개의 구간으로 나누어 'short', 'medium', 'long'으로 라벨링
#그 결과 3개의 구간이 -0.0646 ~ 21.5333, 21.5333 ~ 43.0666, 43.0666 ~ 64.6으로 나뉘게 됨
#retbins=True를 사용하면 카테고리 값과 함께 구간경계값도 함께 반환, false일 경우 카테고리 값만 반환
# retbins=True를 주면 (카테고리 결과, 구간 경계값 배열) 튜플을 반환합니다.
# --> 기존의 데이터(Series)와 구간 경계값 배열을 가진 튜플이 생성됨.



# b = pd.cut(s[passenger_count == 1], 
#        bins=3,
#        labels=['short', 'medium', 'long'],retbins = True)[-1]

# cats_rounded = pd.cut(
#     s[passenger_count == 1],
#     bins=b,
#     labels=['short', 'medium', 'long'],
#     include_lowest=True
# )
# cats_rounded.value_counts()

0
short     7179
medium      26
long         2
Name: count, dtype: int64

In [64]:
pd.cut(s[passenger_count == 1], 
       bins=3,
       labels=['short', 'medium', 'long']).value_counts()
#생성된 구간마다 short , medium, long에 해당하는 데이터 개수를 value_counts()로 계산

0
short     7179
medium      26
long         2
Name: count, dtype: int64

`pd.cut` took the interval from `s.min()` to `s.max()`, divided it into three equal parts, and assigned those to be `short`, `medium`, and `long`. We can see, though, that this meant our `long` category is from 43 miles to 64.6 miles -- numerically one-third of the values' interval, but only including a handful of values!


pd.cut은 s.min()부터 s.max()까지의 구간을 가져와 이를 3개의 동일한 구간으로 나눈 뒤, 각각을 short, medium, long으로 할당했다.
하지만 이렇게 나누면 long 카테고리는 43마일에서 64.6마일까지가 되는데, 이는 값 범위의 3분의 1에 해당하지만 실제 포함되는 값은 몇 개에 불과하다.

문제 원인: 구간 길이만 균등하게 나누고, 데이터 개수는 고려하지 않음

결과: long 구간은 길지만 값이 적어, 분석과 시각화에서 불균형 발생